# Voice leading reinforcement learning agents.

## Introduction.
[...]

$$
\mathbb{E}[\text{return}|\alpha, s]
\ =\ 
\text{reward}+\underset{\ \beta\ \in\ \mathcal{A}_{\alpha(s)}\!\!}{\text{max}}\mathbb{E}\left[\text{return}|\beta, \alpha(s)\right]
$$

$$
v(\alpha, s)
\ =\ 
R(\alpha)+\underset{\ \beta\ \in\ \mathcal{A}_{\alpha(s)}\!\!}{\text{max}}v\big(\beta, \alpha(s)\big)
$$

In [1]:
import copy

import random
import math
import numpy as np
from inspect import isfunction

import torch
import torch.nn as nn

from tqdm import tqdm

___

## Classes for various aspects of music theory.
The various Python classes we define in this section collect important aspects of music theory relevant to problem of voice leading. Using MIDI standard encoding for instance, every note in the scale can be assigned an integer value between $0$ and $127$. In this way, a solution to any voice leading problem can be encoded completely numerically. However, the reward functions for the sequence of step-by-step actions that constitute a proposed solution to a voice leading problem depend on musical theoretical considerations. We will use the classes we define in the present section in order to evaluation remards for our agent's actions.

### Classes related to harmony and melody.

#### Class: `Notes`
Parent(s): *none*

Constructor arguments: *none*

In [2]:
class Notes():
    def __init__(self):
        
        valmod12_to_class = {0: ('C', 'C'),
            1: ('C♯', 'D♭'),
            2: ('D', 'D'),
            3: ('D♯', 'E♭'),
            4: ('E', 'E'),
            5: ('F', 'F'),
            6: ('F♯', 'G♭'),
            7: ('G', 'G'),
            8: ('G♯', 'A♭'),
            9: ('A', 'A'),
            10: ('A♯', 'B♭'),
            11: ('B', 'B')}
        self.valmod12_to_class = valmod12_to_class
        
        all_note_class_names = []
        for key in self.valmod12_to_class:
            class_pair = self.valmod12_to_class[key]
            all_note_class_names.append(class_pair[0])
            all_note_class_names.append(class_pair[1])
        self.all_note_class_names = all_note_class_names
        
        class_to_valmod12 = {}
        for key in self.valmod12_to_class:
            class_pair = self.valmod12_to_class[key]
            for entry in class_pair:
                class_to_valmod12.update({entry: key})
        self.class_to_valmod12 = class_to_valmod12
        
        value_to_class = {}
        for value in range(128):
            valmod12 = value%12
            class_pair = self.valmod12_to_class[valmod12]
            value_to_class.update({value: class_pair})
        self.value_to_class = value_to_class
        
        note_to_value = {}
        for value in self.value_to_class:
            class_pair = self.value_to_class[value]
            sharp_class = class_pair[0]
            flat_class = class_pair[1]
            valmod12 = value%12
            octave = -1 + int((value - valmod12)/12)
            note_to_value.update({sharp_class+'{}'.format(octave): value})
            note_to_value.update({flat_class+'{}'.format(octave): value})
        self.note_to_value = note_to_value

Testing:

In [3]:
notes = Notes()
print(notes.valmod12_to_class[8][0] == 'G♯')
print(notes.class_to_valmod12['E♭'] == 3)
print(notes.value_to_class[54] == ('F♯', 'G♭'))
print(notes.note_to_value['E♭2'] == 39)

True
True
True
True


#### Class: `Scales`
Parent(s):

Constructor arguments: *none*

In [4]:
class Scales():
    def __init__(self):
        
        # Construct modern mode degrees, ascending and descending, as attributes:
        self.long_step_sequence = [2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2]
        
        self.mode_start = {'Ionian': 0,
            'Dorian': 1,
            'Phrygian': 2,
            'Lydian': 3,
            'Mixolydian': 4,
            'Aeolian': 5,
            'Locrian': 6}
        
        modern_mode_steps = {}
        for key, value in self.mode_start.items():
            mode = key
            start_position = value
            current_mode_steps = [self.long_step_sequence[i] for i in range(start_position, start_position+7)]
            modern_mode_steps.update({mode: current_mode_steps})
        self.modern_mode_steps = modern_mode_steps
        
        updown_mode_degrees = {}
        for key, value in self.modern_mode_steps.items():
            mode = key
            step_sequence = value
            degree_sequence = [0]
            for i, step in enumerate(step_sequence):
                scale_degree = degree_sequence[i]
                new_scale_degree = (scale_degree + step)%12
                degree_sequence.append(new_scale_degree)
                rev_degree_sequence = degree_sequence[::-1]
            updown_mode_degrees.update({mode: {'up': copy.deepcopy(degree_sequence),
                                     'down': copy.deepcopy(rev_degree_sequence)}})
        
        # Construct Major mode degrees, ascending and descending, as attributes:
        major_updown = updown_mode_degrees['Ionian']
        updown_mode_degrees.update({'Major': copy.deepcopy(major_updown)})

        # Construct Natural minor mode degrees, ascending and descending, as attributes:
        natural_minor_updown = updown_mode_degrees['Aeolian']
        updown_mode_degrees.update({'Natural_minor': copy.deepcopy(natural_minor_updown)})

        # Construct Harmonic minor mode degrees, ascending and descending, as attributes:
        harmonic_minor_steps = [2, 1, 2, 2, 1, 3, 1]
        harmonic_minor_degree_sequence = [0]
        for i, step in enumerate(harmonic_minor_steps):
            scale_degree = harmonic_minor_degree_sequence[i]
            new_scale_degree = (scale_degree + step)%12
            harmonic_minor_degree_sequence.append(new_scale_degree)
            rev_harmonic_minor_degree_sequence = harmonic_minor_degree_sequence[::-1]
        updown_mode_degrees.update({'Harmonic_minor': {'up': copy.deepcopy(harmonic_minor_degree_sequence),
                                                     'down': copy.deepcopy(rev_harmonic_minor_degree_sequence)}})

        # Construct Melodic minor mode degrees, ascending and descending, as attributes:
        melodic_minor_steps_up = [2, 2, 1, 2, 2, 2, 1]
        melodic_minor_degrees_up = [0]
        for i in range(7):
            current_degree = melodic_minor_degrees_up[i]
            next_degree = (current_degree + melodic_minor_steps_up[i])%12
            melodic_minor_degrees_up.append(next_degree)
        melodic_minor_steps_down = [2, 2, 1, 2, 1, 2, 2]
        melodic_minor_degrees_down = [0]
        for i in range(7):
            current_degree = melodic_minor_degrees_down[i]
            next_degree = (current_degree - melodic_minor_steps_down[i])%12
            melodic_minor_degrees_down.append(next_degree)
        updown_mode_degrees.update({'Melodic_minor': {'up': copy.deepcopy(melodic_minor_degrees_up),
                                                    'down': copy.deepcopy(melodic_minor_degrees_down)}})

        # Combine all ascending and descending mode degrees into attribute dictionary:
        self.updown_mode_degrees = updown_mode_degrees

        # Collect all modes constructed as list attribute:
        mode_list = [key for key in self.updown_mode_degrees]
        self.mode_list = mode_list
 
    # Method for querying the ascending/descending mode degree dictionary attribute:
    def updown_degrees(self, mode):
        assert mode in self.mode_list
        output = self.updown_mode_degrees[mode]
        return output
        

Testing:

In [5]:
scales = Scales()

#### Class: `Key`
Parent(s):

Constructor arguments:
* *root* = `'C'`, 
* *mode* = `'Major'`

**Important.** The constructor for the `Key` class constructs an instance each of the `Notes` and `Scales` classes as attributes of `Key`.

In [6]:
class Key():
    def __init__(self,
                 root = 'C',
                 mode = 'Major'):
        
        self.notes = Notes()
        self.scales = Scales()
        
        assert root in self.notes.all_note_class_names
        assert mode in self.scales.mode_list
        
        self.root_class = root
        self.root_valmod12 = self.notes.class_to_valmod12[self.root_class]
        
        self.mode = mode
        
        self.scale_degrees = self.scales.updown_degrees(mode = self.mode)
        self.up_degrees = self.scale_degrees['up']
        self.triad_degrees = [self.up_degrees[i] for i in [0,2,4]]
        self.triad_valsmod12 = [(self.root_valmod12 + degree)%12 for degree in self.triad_degrees]

Testing:

In [7]:
key = Key(root = 'E', mode = 'Melodic_minor')
print(key.triad_degrees == [0, 4, 7])
print(key.triad_valsmod12 == [4, 8, 11])

True
True


## Classes for rewards.

### Classes for progress-to-final-interval rewards.

#### Class: `SmallProgtoFinScheme`
Parent(s): *none*

Constructor arguments:

In [8]:
class SmallProgtoFinSchema():
    def __init__(self):
        pass
        
    def reward(self,
        chord_0 = np.array([73, 76]),
        chord_1 = np.array([72, 74]),
        final_chord = np.array([35, 45])):
        
        assert len(chord_0) == len(chord_1) == len(final_chord)
        
        n = len(final_chord)
        
        centroid_0 = np.sum(chord_0)/n
        centroid_1 = np.sum(chord_1)/n
        final_centroid = np.sum(final_chord)/n
        
        needed_change = final_centroid - centroid_0
        #needed_direction = np.sign(needed_change)
        
        actual_change = centroid_1 - centroid_0
        #actual_direction = np.sign(actual_change)
        
        if actual_change == 0.:
            change_ratio = -100.
        else:
            change_ratio = needed_change/actual_change
        
        return change_ratio

Testing:

In [9]:
small_prog_to_fin_scheme = SmallProgtoFinSchema()

print(small_prog_to_fin_scheme.reward(chord_0 = np.array([73, 76]),
                                      chord_1 = np.array([73, 75]),
                                      final_chord = np.array([35, 45])))

print(small_prog_to_fin_scheme.reward(chord_0 = np.array([73, 76]),
                                      chord_1 = np.array([73, 75]),
                                      final_chord = np.array([73, 77])))

print(small_prog_to_fin_scheme.reward(chord_0 = np.array([73, 76]),
                                      chord_1 = np.array([73, 76]),
                                      final_chord = np.array([74, 78])))

69.0
-1.0
-100.0


___

## Agent classes.

#### Function: `randinterval`
Arguments: *none*

**Remark.** It occurs to me that I was introducing inductive biases into the agent with the way I had this `randchord` function written. I was randomly selecting an `np.array` $[i_0, i_1, \dots, i_{n-1}]$ by selecting $i_0$, then selecting $i_1>i_0$, and so on, within a given range. With a bit of thought though, it becomes clear that this method is not *independent and identically distributed* (iid). For instance, if we draw a pair or integers $i_0 < i_1$ from the set $[0,1,2]$ using this method, then the probability density ends up being
$$\rho([0,1])=\tfrac{1}{4},\ \ \ \ \ \ \rho([0,2])=\tfrac{1}{4},\ \ \ \ \ \ \text{and}\ \ \ \ \ \ \rho([1,2])=\tfrac{1}{2}.$$

In [10]:
class RandomChord():
    def __init__(self, chord_size = 3, lower_limit = 0, upper_limit = 127):
        assert isinstance(chord_size, int)
        assert chord_size > 0
        assert isinstance(lower_limit, int)
        assert isinstance(upper_limit, int)
        assert lower_limit <= upper_limit
        assert chord_size <= upper_limit - lower_limit + 1
        
        self.chord_size = chord_size
        self.lower_limit = lower_limit
        self.upper_limit = upper_limit
        
        admissible_chords = [[k] for k in range(self.lower_limit, self.upper_limit + 2 - self.chord_size)]
        for i in range(1, self.chord_size):
            new_admissible_chords = []
            for running_chord in admissible_chords:
                new_lower_limit = running_chord[-1] + 1
                for k in range(new_lower_limit, self.upper_limit + 2 - self.chord_size + i):
                    new_chord = running_chord + [k] 
                    new_admissible_chords.append(new_chord)
            admissible_chords = new_admissible_chords
        self.admissible_chords = admissible_chords
        
    
    def sample(self):
        chord = random.choice(self.admissible_chords)
        return chord

Testing:

In [11]:
random_chord = RandomChord(chord_size = 3, lower_limit = 0, upper_limit = 5)

for i in range(4):
    output = random_chord.sample()
    print(output[0]<output[1]<output[2])

True
True
True
True


#### Class: `ActionValue_Spec1`
Parent(s): `torch.nn.Module`

Constructor arguments:
* *layer_count* = `6`
* *layer_features* = `1000`

**Remark: `ReLU` versus `Softmax`.** Because we're implicitly using the *greedy policy*, which, at each state $s$, always selects the action $\alpha$ that maximizes the action-value $v_{\text{greed}}(s,\alpha)$, it might appear that the neural network that approximates $v_{\text{greed}}(s,\alpha)$ should use *softmax* activation at its final layer. However, the specific value of $v_{\text{greed}}(s,\alpha)$ is also important. This activation function $v_{\text{greed}}(s,\alpha)$ is supposed to output the *excpected return* $\mathbb{E}_{\text{greed}}[G|\alpha,\pi]$, which is a (potentially weighted) sum of all future rewards that the agent will obtain under the greedy policy. Because we've already specified our rewards implicitly in the various reward functions we defined above, we will run into trouble if we use softmax. Indeed, $0\le \text{softmax}(x)\le 1$, whereas our reward functions can tske all sorts of integer values, sometimes negative. Thus is makes more sense to use `ReLU` or `LeakyReLU` for activation in our neural network.

In [12]:
def onehot_tensor(index, length):
    assert isinstance(index, int)
    assert isinstance(length, int)
    assert 0 <= index <= length
    
    onehot = torch.Tensor([float(i == index) for i in range(length)])
    
    return onehot

In [13]:
class ActionValue_01(nn.Module):
    def __init__(self,
                 chord_size = 3,
                 lower_limit = 0,
                 upper_limit = 5,
                 layer_count = 8,
                 layer_features = 1000):
        super().__init__()
        
        self.chord_size = chord_size
        self.lower_limit = lower_limit
        self.upper_limit = upper_limit
        
        self.random_chord = RandomChord(chord_size = self.chord_size,
                                        lower_limit = self.lower_limit,
                                        upper_limit = self.upper_limit)
        
        self.admissible_chord_count = len(self.random_chord.admissible_chords)
        
        self.index_to_chord = {i: chord for i, chord in enumerate(self.random_chord.admissible_chords)}
        self.chord_to_index = {tuple(chord): i for i, chord in enumerate(self.random_chord.admissible_chords)}
        self.chord_to_tensor = {tuple(chord): onehot_tensor(index, self.admissible_chord_count) \
                                for index, chord in enumerate(self.random_chord.admissible_chords)}
        
        
        assert isinstance(layer_count, int)
        assert layer_count > 0
        assert isinstance(layer_features, int)
        assert layer_features > 0
        
        self.layer_count = layer_count
        self.layer_features = layer_features
        
        self.layers = nn.ModuleList()
        # Critical here: the `3` in our `in_features = 3 * self.admissible_chord_count` comes from:
        # 0: current state
        # 1: next state
        # 2: final (goal) state
        # The action variable here is implict in the assignment `step current state ← next state`
        self.layers.append(nn.Linear(in_features = 3 * self.admissible_chord_count, out_features = self.layer_features))
        for k in range(self.layer_count-2):
            self.layers.append(nn.Linear(in_features = self.layer_features, out_features = self.layer_features))
        self.layers.append(nn.Linear(in_features = self.layer_features, out_features = 1))
        
        self.activation = nn.ELU()
        
        
    def forward(self, x):
        features = x
        for i, layer in enumerate(self.layers):
            #print(i)
            activated_features = self.activation(features)
            features = layer(activated_features)
            
        return features
    
    
    def greedy_action(self, chord_0, final_chord):
        assert isinstance(chord_0, list)
        assert isinstance(final_chord, list)
        assert len(chord_0) == len(final_chord) == self.random_chord.chord_size
        for i in range(self.random_chord.chord_size):
            assert isinstance(chord_0[i], int)
            assert self.random_chord.lower_limit <= chord_0[i] <= self.random_chord.upper_limit
            assert isinstance(final_chord[i], int)
            assert self.random_chord.lower_limit <= final_chord[i] <= self.random_chord.upper_limit
            
        onehot_0 = self.chord_to_tensor[tuple(chord_0)]
        final_onehot = self.chord_to_tensor[tuple(final_chord)]

        max_index = -math.inf
        for index in range(self.admissible_chord_count):
            test_chord = self.index_to_chord[index]
            test_onehot = self.chord_to_tensor[tuple(test_chord)]
            
            model_input = torch.cat((onehot_0,
                                     test_onehot,
                                     final_onehot))
            
            model_output = self.forward(model_input)
            
            if model_output.item() > max_index:
                max_index = model_output.item()
                chord_that_maximizes = test_chord
                
        return chord_that_maximizes
    
    
    def epsilon_greedy_action(self, epsilon, chord_0, final_chord):
        assert isinstance(epsilon, float)
        assert 0.0 <= epsilon <= 1.0
        
        greedy_or_random = np.random.choice(['greedy', 'random'], p=[1-epsilon, epsilon])
        
        if greedy_or_random == 'greedy':
            output_chord = self.greedy_action(chord_0, final_chord)
        elif greedy_or_random == 'random':
            output_chord = self.random_chord.sample()
        
        return output_chord

Testing:

In [14]:
action_value = ActionValue_01(chord_size = 3,
                              lower_limit = 36,
                              upper_limit = 60,
                              layer_count = 4,
                              layer_features = 500)

x = torch.rand(3 * action_value.admissible_chord_count)
print(action_value(x))

g_a = action_value.greedy_action([36, 38, 40], [37, 39, 44])
print(g_a)

trial_count = 20
eps = 0
for i in tqdm(range(trial_count)):
    e_g_a = action_value.epsilon_greedy_action(.1, [36, 38, 40], [37, 39, 44])
    if e_g_a == g_a:
        eps += 1
print(1-(eps/trial_count))

tensor([0.0072], grad_fn=<AddBackward0>)
[36, 54, 57]


100%|███████████████████████████████████████████| 20/20 [00:22<00:00,  1.13s/it]

0.050000000000000044


### "First species" voice leading reinforcement learning agent.

#### Class: `Agent_01`

Constructor arguments:
* *action_value* = `ActionValue_01`, 
* *start_chord* = `[0, 1, 2]`, 
* *end_chord* = `[3, 4, 5]`, 

**What it does.** 

**Remark: Tips for future `Agent_##`s.** I

In [15]:
class Agent_01():
    def __init__(self,
                 action_value = ActionValue_01(),
                 start_chord = [0, 1, 2],
                 final_chord = [3, 4, 5]):
        
        assert isinstance(action_value, ActionValue_01)
        
        self.action_value = action_value
        
        self.chord_size = self.action_value.random_chord.chord_size
        
        self.lower_limit = self.action_value.random_chord.lower_limit
        self.upper_limit = self.action_value.random_chord.upper_limit
        
        self.chord_to_tensor = self.action_value.chord_to_tensor
        
        assert isinstance(start_chord, list)
        assert isinstance(final_chord, list)
        assert len(start_chord) == len(final_chord) == self.chord_size
        for i in range(self.chord_size):
            assert isinstance(start_chord[i], int)
            assert isinstance(final_chord[i], int)
            assert self.lower_limit <= start_chord[i] <= self.upper_limit
            assert self.lower_limit <= final_chord[i] <= self.upper_limit
        
        self.start_chord = start_chord
        self.start_tensor = self.chord_to_tensor[tuple(self.start_chord)]
        
        self.final_chord = final_chord
        self.final_tensor = self.chord_to_tensor[tuple(self.final_chord)]
        
        self.chord_episode = [self.start_chord]
        self.tensor_episode = [self.start_tensor]
        
        self.small_prog_to_fin_scheme = SmallProgtoFinSchema()
        
        
    def next_interval(self, epsilon = 0.1):
        chord_0 = self.chord_episode[-1]
        
        next_chord = action_value.epsilon_greedy_action(epsilon, chord_0, self.final_chord)
        next_tensor = self.chord_to_tensor[tuple(next_chord)]
        
        self.chord_episode.append(next_chord)
        self.tensor_episode.append(next_tensor)
    

    def last_reward(self):
        assert len(self.chord_episode) > 1
        
        last_chord = self.chord_episode[-2]
        last_action = self.chord_episode[-1]
        
        if last_action == self.final_chord:
            reward = 1000.0
        else:
            reward = self.small_prog_to_fin_scheme.reward(chord_0 = np.array(last_chord),
                                                          chord_1 = np.array(last_action),
                                                          final_chord = np.array(self.final_chord))
            
        return reward

Testing:

In [16]:
action_value = ActionValue_01(chord_size = 3,
                              lower_limit = 36,
                              upper_limit = 60,
                              layer_count = 4,
                              layer_features = 500)

agent = Agent_01(action_value = action_value,
                 start_chord = [36, 38, 40],
                 final_chord = [37, 39, 44])

print(agent.tensor_episode, '\n')

agent.next_interval()
print(agent.tensor_episode)
print(agent.last_reward(), '\n')

agent.next_interval()
print(agent.tensor_episode)
print(agent.last_reward(), '\n')

agent.next_interval()
print(agent.tensor_episode)
print(agent.last_reward(), '\n')

agent.next_interval()
print(agent.tensor_episode)
print(agent.last_reward(), '\n')

[tensor([0., 0., 0.,  ..., 0., 0., 0.])] 

[tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.])]
0.2 

[tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.])]
-100.0 

[tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.])]
-100.0 

[tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., 0., 0., 0.])]
-100.0 



___

## Training loop(s).

### Attempt 1.
#### Verdict(s): 

In [28]:
action_value = ActionValue_01(chord_size = 1,
                              lower_limit = 36,
                              upper_limit = 48,
                              layer_count = 5,
                              layer_features = 100)

print(action_value.random_chord.admissible_chords)
print(len(action_value.random_chord.admissible_chords))

agent = Agent_01(action_value = action_value,
                 start_chord = [36],
                 final_chord = [39])

[[36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48]]
13


In [49]:
episode_count = 10000
present_bias = 1.0
learning_rate = 0.0001

epsilon = 0.25

loss_function = nn.MSELoss().double()

# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(action_value.parameters(), lr = learning_rate)

goal_count = 0
total_actions = 0

for episode_number in tqdm(range(episode_count)):
    
    # print('\n________________________________________________________________________\n')
    
    print('Epsiode count:', episode_number, '\n')
    
    random_starting_chord = action_value.random_chord.sample()
    starting_tensor = action_value.chord_to_tensor[tuple(random_starting_chord)]
    
    agent.chord_episode = [random_starting_chord]
    agent.tensor_episode = [starting_tensor]
    
    agent.final_chord = action_value.random_chord.sample()
    agent.final_tensor = agent.chord_to_tensor[tuple(agent.final_chord)]
    
    episode_loss = 0.0
    action_count = 0
    
    reward_list = []
    
    while (agent.chord_episode[-1] != agent.final_chord) and (len(agent.chord_episode) <= 1000):
        # print('Starting chord:', random_starting_chord, '   ', 'Final goal chord:', agent.final_chord)
        # print('Running state count:', len(agent.chord_episode))
        
        agent.next_interval(epsilon = epsilon)
        
        reward = torch.Tensor([agent.last_reward()])
        reward_list.append(reward)
        
        action_count += 1

        if len(agent.chord_episode) >= 3:
            
            previous_tensor_state = agent.tensor_episode[-3]
            present_tensor_state = agent.tensor_episode[-2]
            next_tensor_state = agent.tensor_episode[-1]
            
            # print('Current chord window:', [agent.chord_episode[-3],
            #            agent.chord_episode[-2],
            #            agent.chord_episode[-1]])
            
            #print(len(previous_tensor_state))
            #print(len(present_tensor_state))
            #print(len(agent.final_tensor))
            
            previous_expected_return = action_value(torch.cat([previous_tensor_state,
                                                               present_tensor_state,
                                                               agent.final_tensor]))
            
            present_expected_return = action_value(torch.cat([present_tensor_state,
                                                              next_tensor_state,
                                                              agent.final_tensor]))
            present_expected_return = torch.Tensor([present_expected_return.item()])
            
            present_reward = reward_list[-1]
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # Compute the loss and its gradients
            loss = loss_function(present_reward + present_bias * present_expected_return,
                                 previous_expected_return).double()
           
            episode_loss += loss.item()

            loss.backward()

            # Adjust learning weights
            optimizer.step()
            
            # print('Loss at this action:', loss.item(), '\n')
            

    if (agent.chord_episode[-1] == agent.final_chord) and len(agent.chord_episode)>1:
        
        goal_count += 1
        
        print('**********************************\n',)
        
        present_tensor_state = agent.tensor_episode[-2]
        next_tensor_state = agent.tensor_episode[-1]

        present_expected_return = action_value(torch.cat([present_tensor_state,
                                                          next_tensor_state,
                                                          agent.final_tensor]))

        present_reward = reward_list[-1]
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute the loss and its gradients
        loss = loss_function(present_reward, present_expected_return).double()

        episode_loss += loss.item()

        loss.backward()

        # Adjust learning weights
        optimizer.step()
            
        # print('Loss at this action:', loss.item(), '\n')
    
    total_actions += action_count
        
    print('Goals per spisode:', goal_count/(1+episode_number))
    print('Average loss:', episode_loss/(1+action_count))    
    print('Actions per episode:', total_actions/(1+episode_number))    
        
     
    # if episode_number%100 == 0:
    #     print('Total loss this episode:', episode_loss)
    #print('\n', reward_list)


  0%|                                         | 1/10000 [00:00<26:59,  6.17it/s]

Epsiode count: 0 

**********************************

Goals per spisode: 1.0
Average loss: 78802.21589396788
Actions per episode: 83.0
Epsiode count: 1 

**********************************

Goals per spisode: 1.0
Average loss: 1951445.5
Actions per episode: 42.5
Epsiode count: 2 



  0%|                                         | 4/10000 [00:00<11:41, 14.25it/s]

**********************************

Goals per spisode: 1.0
Average loss: 151404.3881854672
Actions per episode: 41.333333333333336
Epsiode count: 3 

**********************************

Goals per spisode: 1.0
Average loss: 154474.00364783642
Actions per episode: 40.25
Epsiode count: 4 

**********************************

Goals per spisode: 1.0
Average loss: 346025.3376459414
Actions per episode: 35.2
Epsiode count: 5 

**********************************

Goals per spisode: 1.0
Average loss: 376974.66891334736
Actions per episode: 31.5
Epsiode count: 6 



  0%|                                         | 9/10000 [00:00<10:57, 15.21it/s]

...

  2%|▋                                      | 180/10000 [00:14<14:26, 11.34it/s]

**********************************

Goals per spisode: 0.9157303370786517
Average loss: 114017.7116472327
Actions per episode: 44.51123595505618
Epsiode count: 178 

**********************************

Goals per spisode: 0.9162011173184358
Average loss: 391508.43792205834
Actions per episode: 44.324022346368714
Epsiode count: 179 

**********************************

Goals per spisode: 0.9166666666666666
Average loss: 59411.54419159456
Actions per episode: 44.583333333333336
Epsiode count: 180 



  2%|▋                                      | 182/10000 [00:14<12:59, 12.59it/s]

**********************************

Goals per spisode: 0.9171270718232044
Average loss: 395748.51620596397
Actions per episode: 44.39779005524862
Epsiode count: 181 

**********************************

Goals per spisode: 0.9175824175824175
Average loss: 99978.24784483777
Actions per episode: 44.41208791208791
Epsiode count: 182 



KeyboardInterrupt: 